In [23]:
import numpy as np
import pandas as pd
import seaborn as sns
import re
from statistics import mean
%matplotlib inline

In [86]:
dataset = pd.read_csv('./Diabetes_Data_1999_2008.csv')
admission_src = pd.read_csv('./Admission_Source_Mapping.csv')
admission_type = pd.read_csv('./Admission_Type_Mapping.csv')
discharge_disposition = pd.read_csv('./Discharge_Disposition_Mapping.csv')

print(dataset["metformin-pioglitazone"].unique())

['No' 'Steady']


In [75]:
cleaned_df = dataset[dataset.readmitted == "NO"] #only looking at first encounters
cleaned_df = cleaned_df[cleaned_df.discharge_disposition_id != {18, 19, 20}]

admission_source = cleaned_df.iloc[:, 8:9]
timeHosp = cleaned_df.iloc[:, 9:10]
num_proc = cleaned_df.iloc[:, 12:14]
cleaned_df = cleaned_df.iloc[:, :24]
columns = cleaned_df.iloc[:, 22:24]
numMed = cleaned_df.iloc[:, 14:15]
demographics_df = cleaned_df.iloc[:, 2:5]
diag1_df = cleaned_df.iloc[:, 18:19]
cleaned_df = pd.concat([numMed, diag1_df, demographics_df, admission_source, timeHosp, num_proc, columns], axis = 1, join = 'inner')
cleaned_df = cleaned_df[cleaned_df.race != "?"]
cleaned_df = cleaned_df[cleaned_df.gender != "Unknown/Invalid"]

admin_srcMap = {}
admin_List = cleaned_df["admission_source_id"].value_counts().index.tolist()[:5]

for i, v in enumerate(admin_List):
    admin_srcMap[v] = i
for i in cleaned_df.admission_source_id:
    if i not in admin_srcMap:
        admin_srcMap[i] = 5

cleaned_df["admission_source_id"] = cleaned_df["admission_source_id"].map(admin_srcMap)

diag_1Map = {}
diag_1List = cleaned_df["diag_1"].value_counts().index.tolist()[:50]
diag_1Count = cleaned_df["diag_1"].value_counts().values.tolist()[:50]

for i, v in enumerate(diag_1List):
    diag_1Map[v] = i
for i in cleaned_df.diag_1:
    if i not in diag_1Map:
        diag_1Map[i] = 50

cleaned_df["diag_1"] = cleaned_df["diag_1"].map(diag_1Map)

print(cleaned_df)

        num_medications  diag_1             race  gender      age  \
0                     1      50        Caucasian  Female   [0-10)   
2                    13      50  AfricanAmerican  Female  [20-30)   
3                    16      45        Caucasian    Male  [30-40)   
4                     8      50        Caucasian    Male  [40-50)   
6                    21       0        Caucasian    Male  [60-70)   
...                 ...     ...              ...     ...      ...   
101759               15      21        Caucasian    Male  [80-90)   
101762               18      26  AfricanAmerican  Female  [80-90)   
101763                9      10        Caucasian    Male  [70-80)   
101764               21      11        Caucasian  Female  [80-90)   
101765                3      37        Caucasian    Male  [70-80)   

        admission_source_id  time_in_hospital  num_lab_procedures  \
0                         1                 1                  41   
2                         0      

In [81]:
cleaned_df.to_csv("cleaned_data_v2.csv", index = False)

In [77]:
cumMed = {}
for index, row in cleaned_df.iterrows():
    if row["diag_1"] not in cumMed:
        cumMed[row["diag_1"]] = [row["num_medications"]]
    else:
        cumMed[row["diag_1"]].append(row["num_medications"])
avgMed = {}
for i in cumMed:
    avgMed[i] = round(mean(cumMed[i]), 3)

In [78]:
diagName = pd.read_table('./CMS32_DESC_SHORT_DX.txt', delimiter = "  ")

C:\Users\euhan\Anaconda3\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [79]:
print(diagName)
for col in diagName:
    print(col)

                                   ID                 Diagnosis
0                                0010  Cholera d/t vib cholerae
1                                0011    Cholera d/t vib el tor
2                                0019               Cholera NOS
3                                0020             Typhoid fever
4                                0021       Paratyphoid fever a
...                               ...                       ...
14562  V9129 Quad gest-plac/sac undet                      None
14563    V9190 Mult gest-plac/sac NOS                      None
14564  V9191 Mult gest 2+ monochr NEC                      None
14565  V9192 Mult gest 2+ monoamn NEC                      None
14566  V9199 Mult gest-plac/sac undet                      None

[14567 rows x 2 columns]
ID
Diagnosis


In [80]:
nameMap = {}
print(diagName.ID)
for i in diag_1Map:
    if i != 50:
        print(diagName[diagName.ID == i])
#nameMap[diag_1Map[i]] = diagName.Diagnosis[diagName.ID == i]
print(nameMap)

0                                  0010
1                                  0011
2                                  0019
3                                  0020
4                                  0021
                      ...              
14562    V9129 Quad gest-plac/sac undet
14563      V9190 Mult gest-plac/sac NOS
14564    V9191 Mult gest 2+ monochr NEC
14565    V9192 Mult gest 2+ monoamn NEC
14566    V9199 Mult gest-plac/sac undet
Name: ID, Length: 14567, dtype: object
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
       ID                 Diagnosis
5182  486   Pneumonia, organism NOS
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]


Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
       ID            Diagnosis
3387  340   Multiple sclerosis
Empty Data

Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
       ID           Diagnosis
5074  463   Acute tonsillitis
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
       ID                 Diagnosis
4822  430   Subarachnoid hemorrhage
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
I

Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, Diagnosis]
Index: []
Empty DataFrame
Columns: [ID, D